In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetV2M
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.preprocessing import image

from sklearn.model_selection import train_test_split

import os
import shutil






In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

original_fail_dir = 'FAIL'
original_pass_dir = 'PASS'

base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

os.makedirs(os.path.join(train_dir, 'FAIL'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'PASS'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'FAIL'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'PASS'), exist_ok=True)

fail_images = [os.path.join(original_fail_dir, fname) for fname in os.listdir(original_fail_dir) if fname.endswith(('.jpeg', '.png'))]
pass_images = [os.path.join(original_pass_dir, fname) for fname in os.listdir(original_pass_dir) if fname.endswith(('.jpeg', '.png'))]

fail_train, fail_test = train_test_split(fail_images, test_size=0.2, random_state=42)
pass_train, pass_test = train_test_split(pass_images, test_size=0.2, random_state=42)

def move_files(file_list, target_dir):
    for file_path in file_list:
        shutil.move(file_path, target_dir)

move_files(fail_train, os.path.join(train_dir, 'FAIL'))
move_files(fail_test, os.path.join(test_dir, 'FAIL'))
move_files(pass_train, os.path.join(train_dir, 'PASS'))
move_files(pass_test, os.path.join(test_dir, 'PASS'))

In [ ]:
def mish(x):
    return x * tf.keras.backend.tanh(tf.keras.backend.softplus(x))

train_data_dir = 'data/train'
val_data_dir = 'data/test'
num_epochs = 4

num_classes = len(os.listdir(train_data_dir))

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

def preprocess_input_fn(img):
    return tf.keras.applications.efficientnet_v2.preprocess_input(img)

base_model = EfficientNetV2M(input_shape=(224, 224, 3), include_top=False, weights=None)

for layer in base_model.layers:
    if isinstance(layer, layers.Conv2D):
        layer.kernel_initializer = HeNormal()
        layer.activation = None
        layer.add(layers.BatchNormalization())
        layer.add(layers.Lambda(mish))

base_model.trainable = True

model = models.Sequential([
    layers.InputLayer(shape=(224, 224, 3)),
    layers.Lambda(preprocess_input_fn),
    base_model,
    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=num_epochs
)

val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

Found 647 images belonging to 2 classes.
Found 163 images belonging to 2 classes.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda_1 (Lambda)               │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-m (Functional)   │ (None, 7, 7, 1280)     │    53,150,388 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 62720)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 62720)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │       125,442 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,275,830 (203.23 MB)

 Trainable params: 52,983,798 (202.12 MB)

 Non-trainable params: 292,032 (1.11 MB)

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1275s 56s/step - accuracy: 0.7110 - loss: 0.9831 - val_accuracy: 0.9693 - val_loss: 0.6158
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 2213s 105s/step - accuracy: 0.6856 - loss: 0.7835 - val_accuracy: 0.9693 - val_loss: 0.6104
Epoch 3/50
 5/21 ━━━━━━━━━━━━━━━━━━━━ 23:54 90s/step - accuracy: 0.6356 - loss: 0.8424 

KeyboardInterrupt: 